In [1]:
import os
import pandas as pd
import json as js
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score,log_loss
from sklearn.model_selection import cross_val_score, KFold

# Этап загрузки данных

In [2]:
os.listdir('data')

['sample_submission.csv',
 'matches.jsonlines.bz2',
 'features_test.csv',
 'features.csv',
 'dictionaries',
 'matches_test.jsonlines.bz2',
 'extract_features.py']

In [3]:
df_train = pd.read_csv('data/features.csv', index_col='match_id')
df_test = pd.read_csv('data/features_test.csv', index_col='match_id')

In [4]:
df_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [5]:
df_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


### Удаляем из тренировочного набора информацию  об исходе матча (метрики из будущего)

In [6]:
fut_feat = ['duration','tower_status_radiant', 'tower_status_dire', 'barracks_status_dire', 'barracks_status_radiant']
df_train.drop(columns=fut_feat, axis=1, inplace=True)
df_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,1
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,0


### Посмотрим сколько пропусков в данных

In [7]:
df_train.isnull().sum().sort_values(ascending=False)[:20]

first_blood_player2            43987
radiant_flying_courier_time    27479
dire_flying_courier_time       26098
first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
dire_bottle_time               16143
radiant_bottle_time            15691
radiant_first_ward_time         1836
dire_first_ward_time            1826
radiant_courier_time             692
dire_courier_time                676
d4_hero                            0
d4_level                           0
d4_xp                              0
d3_items                           0
d3_deaths                          0
d3_kills                           0
d3_lh                              0
d4_gold                            0
dtype: int64

### Реализация алгоритма обучения вариант 1

In [8]:
# Заполняю пропуски
df_train.fillna(0, inplace=True)
#Разбиваю сет на X и  y
X = df_train.iloc[:,:-1]
y = df_train.iloc[:,-1]

In [9]:
#Создаем генератор для кросс-валидации
kfold = KFold(shuffle=True)

In [10]:
estimators_list = [10,20,30,35]

for n in estimators_list:
    #Создаем модель
    clf = GradientBoostingClassifier(n_estimators=n)
    # Определяем время старта
    start_time = datetime.datetime.now()
    # Оцениваем точность модели 
    scores = cross_val_score(clf, X, y, cv=kfold, scoring='roc_auc')
    # Выводим результат
    print('Количество деревьев: {}, точность: {:.2f}, время: {}'.format(n, scores.mean(), datetime.datetime.now() - start_time))

Количество деревьев: 10, точность: 0.67, время: 0:00:48.474042
Количество деревьев: 20, точность: 0.68, время: 0:01:34.804506
Количество деревьев: 30, точность: 0.69, время: 0:02:23.355910
Количество деревьев: 35, точность: 0.69, время: 0:02:46.354930


### Ответ на вопрос 1
Всего имеем 12 признаков с пропусками. \
first_blood_time, \
first_blood_team, \
first_blood_player1, \
first_blood_player2, \
radiant_bottle_time, \
radiant_courier_time, \
radiant_flying_courier_time, \
radiant_first_ward_time, \
dire_bottle_time, \
dire_courier_time, \
dire_flying_courier_time, \
dire_first_ward_time 

Полагаю отсутствие значений говорит нам о том что действия описываемого признаком в матче не произошло.\
Возмем признаки dire_courier_time и first_blood_player2

dire_courier_time - время приобретения предмета "courier" \
Пропуски в признаке означает что сторона dire не купила курьера

first_blood_player2 - второй игрок, причастный к событию first_blood\
Пропуски в этом признаке означают, что второй игрок не участвовал в событии first_blood. При чем исходя из данных мы видим что половина случаев first_blood совершались 1-им игроком.
### Ответ на вопрос 2
Целевая переменная в столбце - radiant_win
### Ответ на вопрос 3
Количество деревьев: 30, точность: 0.69, время: 0:02:20.462779
### Ответ на вопрос 4
Больше 30 деревьев смысла использовать нет, так как точность модели после 30 деревьев увеличивается не значительно, однако чем больше деревьев тем больше вероятность переобучить модель
При увеличении количества деревьев для ускорения модели можно уменьшить гиперпараметр learning rate или глубину деревьев. Или как вариант уменьшить размер выборки

### Реализация алгоритма обучения вариант 2

In [11]:
def log_reg (X,y, val_c, kfold):
    """
    Функция для построения модели лог регрессии по заданным параметрам и набору данных
    """
    result = {}
    for c in val_c:
        #Создаем модель
        clf = LogisticRegression(C=c, max_iter=10000)
        # Определяем время старта
        start_time = datetime.datetime.now()
        # Оцениваем точность модели 
        scores = cross_val_score(clf, X, y, cv=kfold, scoring='roc_auc')
        result[c] = scores.mean()
        clf.fit(X,y)        
        # Выводим результат
        print('Параметр С: {:.4f}, точность: {:.2f}, время: {}'.format(c, scores.mean(), datetime.datetime.now() - start_time))
    print('Оптимальный параметр С = {}'.format(max(result, key=result.get)))

In [12]:
#Нормируем признаки
scaler = StandardScaler()
X_scale = pd.DataFrame(scaler.fit_transform(X))
X_scale.columns = X.columns
X_scale.index = X.index
X_scale.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,-2.544364,1.540688,-1.244228,1.400808,1.525972,0.734957,0.969743,-0.537757,-0.578083,-0.509023,...,-0.987486,1.066448,-0.041743,-0.262922,0.640648,0.018054,0.562864,-0.551154,1.846004,-1.121494
1,-2.540452,-0.927798,-0.292258,0.501314,-0.080139,-0.247570,-0.246859,-0.537757,1.017574,1.492930,...,-0.987486,-0.338591,0.578946,-0.262922,0.379585,1.066668,0.562864,0.678170,0.437788,0.043947
2,-2.539231,1.540688,-0.568637,0.501314,0.151070,0.263085,1.190944,-0.537757,-0.578083,1.492930,...,0.391203,-0.823968,-0.824352,0.158654,0.640648,0.018054,0.562864,0.678170,0.437788,0.490286
3,-2.532622,-0.575157,-0.691471,0.501314,0.962950,-0.198013,0.306142,-0.537757,-0.578083,-1.309804,...,-0.987486,-0.594053,0.241615,-0.022021,0.269135,-1.554868,0.562864,-0.551154,-0.970428,0.837439
4,-2.529221,1.540688,-1.182811,0.501314,0.348745,-0.124754,-0.357459,0.968527,-0.578083,-0.108632,...,-0.987486,1.347455,1.024223,-0.022021,0.680811,1.590976,-0.302485,0.678170,-0.970428,-0.228816


### Задание 1

In [13]:
#Оцениваем качество по аналогии с GB
val_C = np.power(10.0, np.arange(-6, 1))
log_reg(X_scale,y, val_C, kfold)

Параметр С: 0.0000, точность: 0.69, время: 0:00:01.409924
Параметр С: 0.0000, точность: 0.70, время: 0:00:01.475667
Параметр С: 0.0001, точность: 0.71, время: 0:00:01.351303
Параметр С: 0.0010, точность: 0.72, время: 0:00:02.609417
Параметр С: 0.0100, точность: 0.72, время: 0:00:03.540821
Параметр С: 0.1000, точность: 0.72, время: 0:00:03.886313
Параметр С: 1.0000, точность: 0.72, время: 0:00:04.022946
Оптимальный параметр С = 1.0


1. Точность выше чем у GB. \
   Параметр С: 1.0000, точность: 0.72, время: 0:00:04.022946\
   Полагаю это связано с линейной зависимостью ответов от признаков \
   Скорость в разы выше чем у GB

### Задание 2

In [14]:
# Удаляем 11 колонок с категориальными признаками
X_scale_out_cat_f = X_scale.drop(columns=[x for x in X if 'hero' in x]+ ['lobby_type'])
X_scale_out_cat_f.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,-2.544364,1.400808,1.525972,0.734957,0.969743,-0.537757,-0.578083,-0.509023,-0.332256,-0.625222,...,-0.987486,1.066448,-0.041743,-0.262922,0.640648,0.018054,0.562864,-0.551154,1.846004,-1.121494
1,-2.540452,0.501314,-0.080139,-0.247570,-0.246859,-0.537757,1.017574,1.492930,0.578881,0.732454,...,-0.987486,-0.338591,0.578946,-0.262922,0.379585,1.066668,0.562864,0.678170,0.437788,0.043947
2,-2.539231,0.501314,0.151070,0.263085,1.190944,-0.537757,-0.578083,1.492930,-0.332256,0.224676,...,0.391203,-0.823968,-0.824352,0.158654,0.640648,0.018054,0.562864,0.678170,0.437788,0.490286
3,-2.532622,0.501314,0.962950,-0.198013,0.306142,-0.537757,-0.578083,-1.309804,-1.243393,-1.170813,...,-0.987486,-0.594053,0.241615,-0.022021,0.269135,-1.554868,0.562864,-0.551154,-0.970428,0.837439
4,-2.529221,0.501314,0.348745,-0.124754,-0.357459,0.968527,-0.578083,-0.108632,-1.243393,-1.008757,...,-0.987486,1.347455,1.024223,-0.022021,0.680811,1.590976,-0.302485,0.678170,-0.970428,-0.228816


In [15]:
#оцениваем качество без категориальных признаков
log_reg(X_scale_out_cat_f,y, val_C, kfold)

Параметр С: 0.0000, точность: 0.69, время: 0:00:01.353248
Параметр С: 0.0000, точность: 0.69, время: 0:00:01.093492
Параметр С: 0.0001, точность: 0.71, время: 0:00:01.297255
Параметр С: 0.0010, точность: 0.72, время: 0:00:02.342174
Параметр С: 0.0100, точность: 0.72, время: 0:00:03.371301
Параметр С: 0.1000, точность: 0.72, время: 0:00:03.428310
Параметр С: 1.0000, точность: 0.72, время: 0:00:03.544275
Оптимальный параметр С = 0.01


2. После удаления категориальных признаков качество не изменилось. Скорее всего это связано с тем что в построенной модели у этих признаков незначительный вес

### Задание 3, 4, 5

In [16]:
#Узнаем количество героев в игре
X.loc[:,[x for x in X if 'hero' in x]].r1_hero.value_counts().index.max()

112

In [17]:
# Создадим новые признаки методом 'Мешок слов'
X_pick = np.zeros((X.shape[0], 112))
for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i, X.loc[match_id, 'r{}_hero'.format(p+1)]-1] = 1
        X_pick[i, X.loc[match_id, 'd{}_hero'.format(p+1)]-1] = -1
X_pick.shape

(97230, 112)

In [18]:
#Создаем дата фрейм для объединения
X_pick = pd.DataFrame(X_pick, index=X_scale_out_cat_f.index)
#Добавлем новый признак
X_new = pd.concat([X_scale_out_cat_f, X_pick], axis=1)
X_new.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,102,103,104,105,106,107,108,109,110,111
match_id,,,,,,,,,,,,,,,,,,,,,
0,-2.544364,1.400808,1.525972,0.734957,0.969743,-0.537757,-0.578083,-0.509023,-0.332256,-0.625222,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-2.540452,0.501314,-0.080139,-0.247570,-0.246859,-0.537757,1.017574,1.492930,0.578881,0.732454,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2.539231,0.501314,0.151070,0.263085,1.190944,-0.537757,-0.578083,1.492930,-0.332256,0.224676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-2.532622,0.501314,0.962950,-0.198013,0.306142,-0.537757,-0.578083,-1.309804,-1.243393,-1.170813,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-2.529221,0.501314,0.348745,-0.124754,-0.357459,0.968527,-0.578083,-0.108632,-1.243393,-1.008757,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#Оцениваем качество с новыми признаками
log_reg(X_new, y, val_C, kfold)

Параметр С: 0.0000, точность: 0.69, время: 0:00:01.853002
Параметр С: 0.0000, точность: 0.70, время: 0:00:01.691133
Параметр С: 0.0001, точность: 0.73, время: 0:00:02.118466
Параметр С: 0.0010, точность: 0.75, время: 0:00:04.147346
Параметр С: 0.0100, точность: 0.75, время: 0:00:07.858203
Параметр С: 0.1000, точность: 0.75, время: 0:00:12.068504
Параметр С: 1.0000, точность: 0.75, время: 0:00:14.976794
Оптимальный параметр С = 1.0


3. В игре 112 различных идентификаторов
5. Качество модели улучшилось, так как новые признаки были учтены моделью и имели достаточный вес

### Задание 6

### Строю модель LogRegression c параметром с=0.1 и добавлением новых признаков связанных с pick-ом

#### 1. Подготовим тестовые данные

In [20]:
#Смотрим пустые значения
df_test.isnull().sum().sort_values(ascending=False)[:15]

first_blood_player2            7766
radiant_flying_courier_time    4885
dire_flying_courier_time       4524
first_blood_time               3552
first_blood_team               3552
first_blood_player1            3552
radiant_bottle_time            2895
dire_bottle_time               2842
radiant_first_ward_time         330
dire_first_ward_time            263
dire_courier_time               130
radiant_courier_time            127
d3_level                          0
d3_lh                             0
d4_gold                           0
dtype: int64

In [21]:
#Заполняем пустые значения
df_test.fillna(0, inplace=True)

In [22]:
# Удаляем категориальные признаки
X_test = df_test.drop(columns=[x for x in df_test if 'hero' in x]+ ['lobby_type'], axis=1)
X_test.shape

(17177, 91)

In [23]:
X_test_scale = pd.DataFrame(scaler.fit_transform(X_test))
X_test_scale.columns = X_test.columns
X_test_scale.index = X_test.index
X_test_scale.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,-2.514875,0.474746,-0.237743,-0.137094,-0.370005,-0.532732,1.003884,0.305165,-0.368985,0.003979,...,-0.992369,0.450016,1.908352,-0.400363,1.149479,0.017743,0.554260,-0.571552,-0.983356,3.200079
7,-2.513895,-1.365630,-1.207913,-1.255660,-1.145090,-0.532732,-0.593200,0.305165,0.567766,0.023887,...,1.788142,-0.574649,0.836945,1.723080,-1.597294,0.017743,-1.177689,-0.571552,0.433564,0.600867
10,-2.512377,-1.365630,-0.862057,-0.742715,-1.145090,-0.532732,-0.593200,1.898546,-1.305736,-1.375081,...,0.397887,-0.399706,-0.817633,-0.466721,0.281014,-1.038655,-0.311714,0.668978,-0.983356,-0.673257
13,-2.508381,-0.445442,-0.938323,-0.537967,-1.145090,0.968865,1.003884,1.101855,-1.305736,-0.920823,...,1.788142,-1.074486,-1.034626,-0.599436,0.523376,0.017743,0.554260,-0.571552,0.433564,-0.469397
16,-2.507087,0.474746,0.038942,-1.059534,-0.812911,-0.532732,-0.593200,0.305165,1.504517,0.945070,...,-0.992369,1.049820,0.999690,-0.400363,0.220423,-1.038655,-0.311714,-0.571552,0.433564,-0.647774


In [24]:
# Создаем мешок слов
X_pick_test = np.zeros((df_test.shape[0], 112))
for i, match_id in enumerate(df_test.index):
    for p in range(5):
        X_pick_test[i, df_test.loc[match_id, 'r{}_hero'.format(p+1)]-1] = 1
        X_pick_test[i, df_test.loc[match_id, 'd{}_hero'.format(p+1)]-1] = -1
X_pick_test.shape

(17177, 112)

In [25]:
#Создаем дата фрейм для объединения
X_pick_test = pd.DataFrame(X_pick_test, index=X_test.index)
#Добавлем новый признак
X_test_new = pd.concat([X_test_scale, X_pick_test], axis=1)
X_test_new.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,102,103,104,105,106,107,108,109,110,111
match_id,,,,,,,,,,,,,,,,,,,,,
6,-2.514875,0.474746,-0.237743,-0.137094,-0.370005,-0.532732,1.003884,0.305165,-0.368985,0.003979,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-2.513895,-1.365630,-1.207913,-1.255660,-1.145090,-0.532732,-0.593200,0.305165,0.567766,0.023887,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,-2.512377,-1.365630,-0.862057,-0.742715,-1.145090,-0.532732,-0.593200,1.898546,-1.305736,-1.375081,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13,-2.508381,-0.445442,-0.938323,-0.537967,-1.145090,0.968865,1.003884,1.101855,-1.305736,-0.920823,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
16,-2.507087,0.474746,0.038942,-1.059534,-0.812911,-0.532732,-0.593200,0.305165,1.504517,0.945070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Строим модель

In [26]:
clf = LogisticRegression(C=0.1, max_iter=10000)
clf.fit(X_new, y)
pred_train = clf.predict_proba(X_new)[:, 1] 
predict_test = clf.predict_proba(X_test_new)[:, 1]
score_train = roc_auc_score(y, pred_train)
print('Точность на обучающей выборке = {:.2f}'.format(score_train))

Точность на обучающей выборке = 0.75


In [27]:
result = pd.DataFrame({'match_id': df_test.index, 'radiant_win': predict_test})

In [28]:
result.head()

,match_id,radiant_win
0,6,0.824659
1,7,0.757215
2,10,0.187795
3,13,0.862392
4,16,0.239980


In [29]:
#минимальное и максимальное значение прогноза
result.radiant_win.agg([min,max])

min    0.008577
max    0.996460
Name: radiant_win, dtype: float64

In [30]:
result.to_csv('my_submissions_1.csv', index=False)

## Отчет по 2 этапу
1. Точность выше чем у GB. \
   Параметр С: 1.0000, точность: 0.72, время: 0:00:04.022946 \
   Полагаю это связано с линейной зависимостью ответов от признаков \
   Скорость в разы выше чем у GB
   
2. После удаления категориальных признаков качество не изменилось. Скорее всего это связано с тем что в построенной модели у этих признаков незначительный вес 

3. В игре 112 различных идентификаторов 

4. При добавлении мешка слов качество модели улучшилось, так как новые признаки были учтены моделью и имели достаточный вес 

5. min    0.008577 \
   max    0.996460 